## Investigate high error rate
When we overfit a model with `sold_price_loss` close to 0, and save and reload the model, we observ a high prediction mean error rate. This notebook focuses on why this is the case.

Lets first retrive a model with zero `sold_price_loss`

In [1]:
import os
import sys
from pathlib import Path
import shutil

import tensorflow as tf
import tensorflow_datasets as tfds
from datetime import datetime

sys.path.append(f'{os.getcwd()}/../../')
from housing_model.models.keras_model import KerasModelTrainer, TrainParams, ModelParams, EarlyStoppingSetting
from housing_model.models.trainer import eval_model_on_tfds
from housing_model.data.tf_housing import TfHousing
from housing_model.data.example import Features, Example
from housing_model.data.tf_housing.feature_names import SOLD_PRICE, MAP_LAT, MAP_LON, LAND_FRONT, LAND_DEPTH, DATE_END


Lets load the model first:

In [2]:
model_path = os.path.abspath('../../model')
assert os.path.exists(model_path),\
    f"cannot find the overfitted model in {model_path}"

keras_model = KerasModelTrainer.load(model_path)

2022-09-09 20:21:41.733017: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


now load the data set (please manually verify the `ml_num`)

In [3]:
train_ds = tfds.load('tf_housing', split='train').take(4).cache()

for d in train_ds: 
    print(d['metadata']['ml_num'])

tf.Tensor(b'206287', shape=(), dtype=string)
tf.Tensor(b'W4450810', shape=(), dtype=string)
tf.Tensor(b'N4595018', shape=(), dtype=string)
tf.Tensor(b'N4525209', shape=(), dtype=string)


2022-09-09 20:21:42.136101: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-09 20:21:42.223521: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Check the loss is 0:

In [4]:
dataset_size = len(list(train_ds))
train_params = TrainParams(
    batch_size=dataset_size, epochs=1, learning_rate=1e-2,
    early_stopping=EarlyStoppingSetting(patience=500)
)
arc_params = keras_model.model_params.arc_params

keras_model.keras_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=train_params.learning_rate),
    loss={
        arc_params.price_feature_name: tf.keras.losses.MeanSquaredLogarithmicError(),
        arc_params.bits_feature_name: tf.keras.losses.BinaryCrossentropy()
    },
    loss_weights={
        arc_params.price_feature_name: 0.99,
        arc_params.bits_feature_name: 0.01
    }
)

hist = keras_model.keras_model.fit(
    keras_model.data_provider.setup_data(train_ds, dataset_size),
    epochs=train_params.epochs
)

print(hist.history['loss'][-1])

1/1 [==============================] - 2s 2s/step - loss: 1.1927 - bits_loss: 88.2753 - sold_price_loss: 0.3130
1.192667841911316


The loss is not zero, was the model saved properly or the problem is in above code, first check if we can overfit the model again:

In [5]:
hist = keras_model.fit_model(
    train_ds,
    None,
    # train_ds.take(dataset_size).cache(),
    TrainParams(
        batch_size=dataset_size, epochs=500, learning_rate=1e-2,
        early_stopping=EarlyStoppingSetting(patience=500)
    ),
)
print(hist.history['loss'][-1])

2022-09-09 20:21:44.661170: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-09-09 20:21:44.661187: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-09-09 20:21:44.661747: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
/Users/majid/git/housing/venv/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 58.0174 - bits_loss: 478.2324 - sold_price_loss: 53.7728
Epoch 2/500
1/1 [==============================] - 0s 35ms/step - loss: 24.9515 - bits_loss: 223.1548 - sold_price_loss: 22.9494
Epoch 3/500
1/1 [==============================] - 0s 4ms/step - loss: 5.2844 - bits_loss: 183.8018 - sold_price_loss: 3.4812
Epoch 4/500
1/1 [==============================] - 0s 4ms/step - loss: 1.2571 - bits_loss: 101.7997 - sold_price_loss: 0.2415
Epoch 5/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7499 - bits_loss: 66.3856 - sold_price_loss: 0.0869
Epoch 6/500
1/1 [==============================] - 0s 4ms/step - loss: 181.7932 - bits_loss: 82.5731 - sold_price_loss: 182.7954


2022-09-09 20:21:46.908965: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-09-09 20:21:46.908985: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-09-09 20:21:46.915673: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-09-09 20:21:46.925776: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-09-09 20:21:46.932543: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/scalars/20220909-202144/train/plugins/profile/2022_09_09_20_21_46

2022-09-09 20:21:46.934767: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/scalars/20220909-202144/train/plugins/profile/2022_09_09_20_21_46/Majids-MBP.trace.json.gz
2022-09-09 20:21:46.941342: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/scalars/20220909-202144/tr

Epoch 7/500
1/1 [==============================] - 0s 4ms/step - loss: 1.2633 - bits_loss: 117.3591 - sold_price_loss: 0.0906
Epoch 8/500
1/1 [==============================] - 0s 3ms/step - loss: 1.3037 - bits_loss: 121.3737 - sold_price_loss: 0.0908
Epoch 9/500
1/1 [==============================] - 0s 4ms/step - loss: 1.2858 - bits_loss: 114.6379 - sold_price_loss: 0.1408
Epoch 10/500
1/1 [==============================] - 0s 4ms/step - loss: 1.1616 - bits_loss: 102.4238 - sold_price_loss: 0.1388
Epoch 11/500
1/1 [==============================] - 0s 4ms/step - loss: 1.1041 - bits_loss: 96.4075 - sold_price_loss: 0.1414
Epoch 12/500
1/1 [==============================] - 0s 4ms/step - loss: 0.9834 - bits_loss: 84.2939 - sold_price_loss: 0.1418
Epoch 13/500
1/1 [==============================] - 0s 5ms/step - loss: 0.8604 - bits_loss: 76.1299 - sold_price_loss: 0.1001
Epoch 14/500
1/1 [==============================] - 0s 4ms/step - loss: 0.7927 - bits_loss: 69.6865 - sold_price_loss

1/1 [==============================] - 0s 5ms/step - loss: 0.0854 - bits_loss: 8.5350 - sold_price_loss: 1.0066e-04
Epoch 72/500
1/1 [==============================] - 0s 4ms/step - loss: 0.0878 - bits_loss: 8.1963 - sold_price_loss: 0.0059
Epoch 73/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0655 - bits_loss: 5.9773 - sold_price_loss: 0.0058
Epoch 74/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1254 - bits_loss: 11.8327 - sold_price_loss: 0.0072
Epoch 75/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1123 - bits_loss: 10.5220 - sold_price_loss: 0.0072
Epoch 76/500
1/1 [==============================] - 0s 4ms/step - loss: 0.0938 - bits_loss: 9.2580 - sold_price_loss: 0.0012
Epoch 77/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1147 - bits_loss: 10.8894 - sold_price_loss: 0.0058
Epoch 78/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1378 - bits_loss: 12.9331 - sold_price_loss: 0.0085
Epoch

...

1/1 [==============================] - 0s 4ms/step - loss: 3.7706e-16 - bits_loss: 3.7706e-14 - sold_price_loss: 0.0000e+00
Epoch 255/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6771e-16 - bits_loss: 3.6771e-14 - sold_price_loss: 0.0000e+00
Epoch 256/500
1/1 [==============================] - 0s 4ms/step - loss: 3.9280e-16 - bits_loss: 3.9280e-14 - sold_price_loss: 0.0000e+00
Epoch 257/500
1/1 [==============================] - 0s 4ms/step - loss: 4.3307e-16 - bits_loss: 4.3307e-14 - sold_price_loss: 0.0000e+00
Epoch 258/500
1/1 [==============================] - 0s 5ms/step - loss: 5.2908e-16 - bits_loss: 5.2908e-14 - sold_price_loss: 0.0000e+00
Epoch 259/500
1/1 [==============================] - 0s 4ms/step - loss: 6.3317e-16 - bits_loss: 6.3317e-14 - sold_price_loss: 0.0000e+00
Epoch 260/500
1/1 [==============================] - 0s 5ms/step - loss: 6.8303e-16 - bits_loss: 6.8303e-14 - sold_price_loss: 0.0000e+00
Epoch 261/500
1/1 [=============================

1/1 [==============================] - 0s 4ms/step - loss: 3.6517e-16 - bits_loss: 3.6517e-14 - sold_price_loss: 0.0000e+00
Epoch 314/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6550e-16 - bits_loss: 3.6550e-14 - sold_price_loss: 0.0000e+00
Epoch 315/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6580e-16 - bits_loss: 3.6580e-14 - sold_price_loss: 0.0000e+00
Epoch 316/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6601e-16 - bits_loss: 3.6601e-14 - sold_price_loss: 0.0000e+00
Epoch 317/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6629e-16 - bits_loss: 3.6629e-14 - sold_price_loss: 0.0000e+00
Epoch 318/500
1/1 [==============================] - 0s 5ms/step - loss: 3.6640e-16 - bits_loss: 3.6640e-14 - sold_price_loss: 0.0000e+00
Epoch 319/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6663e-16 - bits_loss: 3.6663e-14 - sold_price_loss: 0.0000e+00
Epoch 320/500
1/1 [=============================

1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 373/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 374/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 375/500
1/1 [==============================] - 0s 5ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 376/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 377/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 378/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 379/500
1/1 [=============================

1/1 [==============================] - 0s 5ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 432/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 433/500
1/1 [==============================] - 0s 5ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 434/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 435/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 436/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 437/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 438/500
1/1 [=============================

1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 491/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 492/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 493/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 494/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 495/500
1/1 [==============================] - 0s 5ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 496/500
1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
Epoch 497/500
1/1 [=============================

The model overfit agian, now we can check the above code without save/load the model again:

In [6]:
hist = keras_model.keras_model.fit(
    keras_model.data_provider.setup_data(train_ds, dataset_size),
    epochs=train_params.epochs
)

print(hist.history['loss'][-1])

1/1 [==============================] - 0s 4ms/step - loss: 3.6799e-16 - bits_loss: 3.6799e-14 - sold_price_loss: 0.0000e+00
3.6798983757482535e-16


This code worked, please re-run above cell to make sure, everything is the same.

(I have done it and I got zero loss agian)

This suggest the problem is in loading and saving the model. Lets verify it:

In [7]:
tmp_dir = Path('/tmp/tmp_model')
if tmp_dir.exists():
    shutil.rmtree(tmp_dir)
tmp_dir.mkdir()
    
keras_model.save(tmp_dir)
loaded_model = KerasModelTrainer.load(tmp_dir)

for lw, ow in zip(loaded_model.keras_model.weights, keras_model.keras_model.weights):
    print(lw - ow)


tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 128), dtype=float32)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(128,), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

The model weight are the same!

In [8]:
train_params = TrainParams(
    batch_size=dataset_size, epochs=1, learning_rate=1e-2,
    early_stopping=EarlyStoppingSetting(patience=500)
)
arc_params = loaded_model.model_params.arc_params

loaded_model.keras_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=train_params.learning_rate),
    loss={
        arc_params.price_feature_name: tf.keras.losses.MeanSquaredLogarithmicError(),
        arc_params.bits_feature_name: tf.keras.losses.BinaryCrossentropy()
    },
    loss_weights={
        arc_params.price_feature_name: 0.99,
        arc_params.bits_feature_name: 0.01
    }
)
for lw, ow in zip(loaded_model.keras_model.weights, keras_model.keras_model.weights):
    print(lw - ow)



tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 128), dtype=float32)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(128,), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [9]:
res = eval_model_on_tfds(train_ds, keras_model.make_predictor())

/Users/majid/git/housing/venv/lib/python3.7/site-packages/keras/engine/functional.py:585: UserWarning: Input dict contained keys ['sold_price'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


In [10]:
res.value

{'cnt': 4,
 'mean': 0.0,
 'med': 0.0,
 'var': 0.0,
 'outlier_cnt': 0,
 'in_range_cnt': 4,
 'in_range_rate': 1.0,
 'outlier_rate': 0.0}

In [11]:
for x, y in keras_model.data_provider.setup_data(train_ds, dataset_size):
    res = keras_model.keras_model(x)
    print(res['sold_price'].numpy())
    print(y['sold_price'].numpy())

[870000. 458600. 800000. 950000.]
[870000. 458600. 800000. 950000.]


In [12]:
examples = [
    Example(ml_num=ex['metadata']['ml_num'].numpy().decode('UTF-8'), sold_price=int(ex[SOLD_PRICE].numpy().item()), features=Features(
        house_sigma_estimation=0.0,
        map_lat=ex[MAP_LAT].numpy().item(),
        map_lon=ex[MAP_LON].numpy().item(),
        land_front=ex[LAND_FRONT].numpy().item(),
        land_depth=ex[LAND_DEPTH].numpy().item(),
        date_end=datetime.fromtimestamp(
            int(ex[DATE_END].numpy().item() * 24 * 3600) 
            + datetime(1970, 1, 1).timestamp()
        )
    ))
    for ex in train_ds
]

encoded_features = [
    TfHousing.to_features(Example(sold_price=ex.sold_price, ml_num=ex.ml_num, features=ex.features))[1]
    for ex in examples
]

In [13]:
for encoded in encoded_features:
    for original in train_ds:
        if encoded['metadata']['ml_num'] == original['metadata']['ml_num']:
            print(encoded)
            print()
            print(original)
            print()
            print("======")


{'sold_price': 870000, 'map/lat': 44.550228118896484, 'map/lon': -79.4133071899414, 'land/front': 93.41000366210938, 'land/depth': 200.89999389648438, 'date_end': 18079.0, 'metadata': {'ml_num': '206287'}}

{'date_end': <tf.Tensor: shape=(), dtype=float32, numpy=18079.0>, 'land/depth': <tf.Tensor: shape=(), dtype=float32, numpy=200.9>, 'land/front': <tf.Tensor: shape=(), dtype=float32, numpy=93.41>, 'map/lat': <tf.Tensor: shape=(), dtype=float32, numpy=44.55023>, 'map/lon': <tf.Tensor: shape=(), dtype=float32, numpy=-79.41331>, 'metadata': {'ml_num': <tf.Tensor: shape=(), dtype=string, numpy=b'206287'>}, 'sold_price': <tf.Tensor: shape=(), dtype=float32, numpy=870000.0>}

{'sold_price': 458600, 'map/lat': 43.5041389465332, 'map/lon': -79.8794937133789, 'land/front': 1.0, 'land/depth': 1.0, 'date_end': 18045.0, 'metadata': {'ml_num': 'W4450810'}}

{'date_end': <tf.Tensor: shape=(), dtype=float32, numpy=18045.0>, 'land/depth': <tf.Tensor: shape=(), dtype=float32, numpy=1.0>, 'land/front'